In [ ]:
from transformers import AutoImageProcessor, AutoModelForDepthEstimation
import matplotlib.pyplot as plt
from PIL import Image
import torch
import gc

In [ ]:
image_path = '../data/patryk_reka.jpg'
output_path = '../data/outputs'

In [ ]:
image_processor = AutoImageProcessor.from_pretrained("LiheYoung/depth-anything-small-hf")
model = AutoModelForDepthEstimation.from_pretrained("LiheYoung/depth-anything-small-hf")

In [ ]:
# Load and preprocess an image.
image = Image.open(image_path)
inputs = image_processor(images=image, return_tensors="pt")

In [ ]:
# Run inference.
with torch.no_grad():
  outputs = model(**inputs)

In [ ]:
post_processed_output = image_processor.post_process_depth_estimation(
  outputs,
  target_sizes=[(image.height, image.width)],
)

In [ ]:
predicted_depth = post_processed_output[0]["predicted_depth"]
depth = (predicted_depth - predicted_depth.min()) / (predicted_depth.max() - predicted_depth.min())
depth = depth.detach().cpu().numpy() * 255
depth = Image.fromarray(depth.astype("uint8"))


In [ ]:
depth.save(output_path + '/deptha.jpg')

In [ ]:
print("Zwalnianie pamięci zajmowanej przez model...")
del model
del image
gc.collect()
print("Pamięć zwolniona.")

In [ ]:
original_image = Image.open(image_path).convert('RGB').rotate(-90, expand=True)

fig, axs = plt.subplots(1, 2, figsize=(24, 8))
axs[0].imshow(original_image)
axs[0].set_title('Original Image')
axs[0].axis('off')

im1 = axs[1].imshow(depth, cmap='gray')
axs[1].set_title('Grayscale Depth Map')
axs[1].axis('off')
fig.colorbar(im1, ax=axs[1], fraction=0.046, pad=0.04, label='Depth Intensity')

plt.tight_layout()
plt.show()